In [30]:
import numpy as np
import cv2
from imgaug import augmenters as iaa
import imgaug as ia


f = open('train_aug_9.txt','w')
f2 = open('train.txt','r')

ia.seed(90)


seq = iaa.Sequential([ 
    iaa.Multiply((0.8, 1.2), per_channel=0.2),
    # Apply affine transformations to each image.
    # Scale/zoom them, translate/move them, rotate them and shear them.
    iaa.Affine(
        scale={"x": (0.5, 1.0), "y": (0.5, 1.0)},
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
        rotate=(-90, 90),
        shear=(-8, 8)
    )],
    random_order=True)  # apply augmenters in random order

seq = seq.to_deterministic()


while True:
    entire = f2.readline()
    if entire is '':
        break
    sp = entire.split()
    fname = sp[0].split('/')
    
    images = []
    img = cv2.imread(sp[0])
    images.append(img)
    
    coods = []
    
    if len(sp) > 2:
        for i in range(1,len(sp)):
            cood = sp[i].split(',')
            coods.append(cood)
    else:
        cood = sp[-1].split(',')
        coods.append(cood)    
        
#    if len(sp) >2:
    bbs = ia.BoundingBoxesOnImage([
        ia.BoundingBox(x1=coods[i][0], y1=coods[i][1], x2=coods[i][2], y2=coods[i][3],label=coods[i][4]) for i in range(len(sp)-1)
    ], shape=img.shape)

    images_aug = seq.augment_images(images)
    bbs_aug = seq.augment_bounding_boxes([bbs])[0].remove_out_of_image().clip_out_of_image()
    
    cv2.imwrite('aug/%s_9.jpg'%(fname[-1][:-4]),images_aug[0])
    data = 'dataset/%s_9.jpg'%(fname[-1][:-4])
    for i in range(len(bbs_aug.bounding_boxes)):
        test = bbs_aug.bounding_boxes[i]
        data = data + ' %d,%d,%d,%d,%d'%(int(test.x1),int(test.y1),int(test.x2),int(test.y2),int(test.label))
    data = data+'\n'
    f.write(data)

f.close()
f2.close()

In [12]:
f.close()
f2.close()